# (21) Dekel Sharing Stuff

**Motivation**: Device = ```cuda:1``` <br>

In [1]:
# HIDE CODE


import os, sys
from IPython.display import display

# tmp & extras dir
git_dir = os.path.join(os.environ['HOME'], 'Dropbox/git')
extras_dir = os.path.join(git_dir, 'jb-vae/_extras')
fig_base_dir = os.path.join(git_dir, 'jb-vae/figs')
tmp_dir = os.path.join(git_dir, 'jb-vae/tmp')

# GitHub
sys.path.insert(0, os.path.join(git_dir, '_PoissonVAE'))
from analysis.eval import sparse_score
from figures.fighelper import *
from vae.train_vae import *

# warnings, tqdm, & style
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)
from rich.jupyter import print
%matplotlib inline
set_style()

## Device

In [2]:
device_idx = 1
device = f'cuda:{device_idx}'

from analysis.final import sort_fits, analyze_fits, add_nelbo_diff
from figures.table import table_entry_loss
from figures.point import nelbo_point
from analysis.stats import ttest

## Load, make df

### Fit names

In [3]:
root = add_home('Dropbox/chkpts/PoissonVAE')

fits, fits_st, fits_etc = sort_fits()
len(fits), len(fits_st), len(fits_etc)

(460, 90, 527)

In [4]:
def accept_fn(name):
    archi = name.split('-')[1]
    accept = (
        archi in ['<lin|lin>', '<conv+b|lin>']
        and '-b' not in name
    )
    return accept


def accept_fn_etc(name):
    archi = name.split('-')[1]
    accept = (
        archi in ['<lin|lin>', '<conv+b|lin>']
        and any(fn in name for fn in ['relu', 'exp'])
        and 'gaussian' in name
        and '-b' not in name
    )
    return accept


fits, fits_st, fits_etc = sort_fits()

fits = sorted(filter(accept_fn, fits))
fits_st = sorted(filter(accept_fn, fits_st))
fits_etc = sorted(filter(accept_fn_etc, fits_etc))

msg = ' ——— '.join([
    f"# fits: {len(fits)}",
    f"# fits_st: {len(fits_st)}",
    f"# fits_etc: {len(fits_etc)}",
])
print(msg)

# fits: 240 ——— # fits_st: 60 ——— # fits_etc: 60

In [5]:
def accept_fn_dekel(name):
    archi = name.split('-')[1]
    accept = (
        archi == '<conv+b|lin>'
        and '-exact_' not in name
        and 'DOVES' in name
    )
    return accept

fits_dekel = sorted(filter(accept_fn_dekel, fits + fits_etc))
print(f"# fits_dekel {len(fits_dekel)}")

# fits_dekel 30

In [12]:
results = collections.defaultdict(dict)
for name in fits_dekel:
    tr, meta = load_quick(
        pjoin(root, name),
        device=device,
        verbose=False,
        lite=True,
    )
    phi = tonp(tr.model.fc_dec.weight)
    dead = tr.find_dead_neurons()
    results['phi'][name] = phi
    results['dead'][name] = dead
    print(f"name: {name} ——— # dead: {dead.sum()}")

name: categorical-<conv+b|lin>-DOVES-mc_solo-1_(2024_05_16,21:55) ——— # dead: 2

name: categorical-<conv+b|lin>-DOVES-mc_solo-2_(2024_05_17,08:12) ——— # dead: 2

name: categorical-<conv+b|lin>-DOVES-mc_solo-3_(2024_05_17,17:26) ——— # dead: 2

name: categorical-<conv+b|lin>-DOVES-mc_solo-4_(2024_05_14,06:07) ——— # dead: 2

name: categorical-<conv+b|lin>-DOVES-mc_solo-5_(2024_05_14,15:47) ——— # dead: 2

...

name: poisson-<conv+b|lin>-DOVES-mc_solo-1_(2024_05_14,07:13) ——— # dead: 79

name: poisson-<conv+b|lin>-DOVES-mc_solo-2_(2024_05_14,19:37) ——— # dead: 108

name: poisson-<conv+b|lin>-DOVES-mc_solo-3_(2024_05_15,06:56) ——— # dead: 93

name: poisson-<conv+b|lin>-DOVES-mc_solo-4_(2024_05_15,18:21) ——— # dead: 85

name: poisson-<conv+b|lin>-DOVES-mc_solo-5_(2024_05_16,05:45) ——— # dead: 98

In [13]:
save_obj(
    obj=dict(results),
    file_name='phi_nonlin',
    save_dir=tmp_dir,
    mode='npy',
)

[PROGRESS] 'phi_nonlin.npy' saved at
/home/hadi/Dropbox/git/jb-vae/tmp